In [ ]:
import json
import re
import pandas as pd
import openai
from tenacity import retry, wait_exponential, stop_after_attempt
import requests
import base64

# Constants
EXCEL_FILE_PATH = 'links.xlsx'  # Create an excel file with the URL for the Book Cover images, recommended website: postimg.cc
IMAGE_URL_COLUMN = 'url'  # Update with the correct column name containing image URLs
API_KEY = 'YOUR_OPEN_AI_KEY'  # Replace with your actual OpenAI API key

# Set the OpenAI API key
client = openai.OpenAI(api_key=API_KEY)

@retry(wait=wait_exponential(multiplier=1, min=4, max=10), stop=stop_after_attempt(3))
def download_image(image_url: str) -> str:
    """Downloads an image from the given URL and returns the base64-encoded image data."""
    try:
        response = requests.get(image_url)
        image_data = response.content
        base64_image = base64.b64encode(image_data).decode('utf-8')
        return base64_image
    except Exception as e:
        print(f"Error downloading image from {image_url}: {e}")
        raise e

@retry(wait=wait_exponential(multiplier=1, min=4, max=10), stop=stop_after_attempt(3))
def send_image_to_model(base64_image: str, prompt: str) -> dict:
    """Sends a base64-encoded image to the OpenAI model for high-resolution processing and returns the response."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "low"
                            }
                        }
                    ],
                }
            ],
            max_tokens=500,
        )
        print(response)
        return response.choices[0].message.content
    except openai.APIError as e:
        if e.code == 400:
            print(f"Error sending image to OpenAI (status code 400): {e}")
        else:
            print(f"Error sending image to OpenAI: {e}")
        raise e
    except Exception as e:
        print(f"Error sending image to OpenAI: {e}")
        raise e

def gpt_string_to_json(gpt_response: str) -> dict:
    # Remove doubled '{' and '}' characters
    gpt_response = gpt_response.replace("}}", "}").replace("{{", "{")

    # Remove any text before the first '{' character
    first_brace_index = gpt_response.find("{")
    if first_brace_index != -1:
        gpt_response = gpt_response[first_brace_index:]

    # Remove any text after the last '}' character
    last_brace_index = gpt_response.rfind("}")
    if last_brace_index != -1:
        gpt_response = gpt_response[: last_brace_index + 1]

    # Replace newlines within string values with '\n'
    gpt_response = re.sub(
        r'(?<=": ")(.+?)(?=")',
        lambda m: m.group(1).replace("\n", "\\n"),
        gpt_response,
        flags=re.DOTALL,
    )

    # Remove any control characters from the response
    gpt_response = re.sub(r"[\x00-\x09\x0b-\x1f\x7f-\x9f]", "", gpt_response)

    try:
        json_response = json.loads(gpt_response)
        return json_response
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        print(f"Problematic JSON string: {gpt_response}")
        return None

def process_images(excel_path: str, image_url_column: str) -> pd.DataFrame:
    """Processes images listed in an Excel file and adds results to a DataFrame."""
    # Load the Excel file into a DataFrame
    df = pd.read_excel(excel_path)

    # Initialize an empty DataFrame to store the results
    results_df = pd.DataFrame()

    # Loop through each image URL in the DataFrame
    for _, row in df.iterrows():
        image_url = row[image_url_column]

        # Placeholder for user to provide the prompt
        prompt = """
I'm sending an image of a Book Cover:
- I need you to get the full name of the book, 
- if in the book cover there is information about the volume, edition or collection, include it in the name of the book. Example: "Name of the Book" or "Name of the Book (Vol. 1)"
- I also need the name of the Author if present in the cover, else say 'sem autor'.
- You also need to tell me condition of the book, describe it so I can put as the description of the product so I can sell it. They are all used, you can be generic about its condition. These are used books. See if the book cover is in place or if there are any tears. If there are any marks or scratches, mention them. If the book is in perfect condition, you can say that too.

There is another thing, there are several categories of book, I need you to get from this list which number this book belongs to:
História e Culturas - 101565
Políticas, Direito e Ciências Sociais - 101547
Religião e Filosofia - 101564
Literatura Clássica - 101555
Idiomas e Dicionários - 101551
Ficção - 101556
Ação, Crime e Suspense - 101557
Biografias e Memória - 101559
Ciência e Matemática - 101560

All your responses need to be in Brazilian Portuguese.

Your response must be in JSON format, like this:
{"book_name": "the name of the book": "author": "author of the book", "condition": "condition of the book", "category": "category number"}
"""  # Replace with your specific prompt

        # Download the image and get the base64-encoded data
        base64_image = download_image(image_url)

        # Send the base64-encoded image and prompt to the model and get the response
        response_text = send_image_to_model(base64_image, prompt)

        if response_text is None:
            print(f"Skipping image at {image_url} due to API error.")
            continue  # Skip to the next image if there's an API error

        try:
            response_json = gpt_string_to_json(response_text)
        except Exception as e:
            print(f"Error converting GPT response to JSON: {e}")
            continue

        new_row = pd.DataFrame({"url": image_url, "product_name": response_json["book_name"] + " - " + response_json["author"], "condition": response_json["condition"], "category": response_json["category"], "description": "Livro usado." + response_json["condition"]}, index=[0])
        
        # Concatenate the new row to the results DataFrame
        results_df = pd.concat([results_df, new_row], ignore_index=True)

        # Save the current progress after each processed image
        results_df.to_excel('processed_results.xlsx', index=False)

    return results_df

if __name__ == "__main__":
    # Run the image processing
    results = process_images(EXCEL_FILE_PATH, IMAGE_URL_COLUMN)
    print(results)